# Fase 2: Preprocesamiento

## Configuración de la GPU

Por un motivo que se desconoce, cuando se utiliza el acelerador P100, es necesario limitar el crecimiento de la GPU (para más detalles, revisar [acá](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth)). Otro punto es que esta celda debe venir antes de cualquier importación ya que internamente modifican la capacidad de la GPU (ver el siguiente [hilo](https://github.com/hunglc007/tensorflow-yolov4-tflite/issues/171)), y por ende, obtenemos el error `Physical devices cannot be modified after being initialized`.

In [ ]:
from tensorflow.config import list_physical_devices
from tensorflow.config.experimental import set_memory_growth

gpus = list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            set_memory_growth(gpu, True)
    except RuntimeError as e:
        print("error:", e)

## Instalación de dependencias

Utilizaremos las siguientes librerías:

- `dotwiz`: Permite acceder a los diccionarios en Python con una notación de tipo "punto" (exactamente como se hace en Javascript).
- `gdown`: Permite descargar archivos desde Google Drive.

In [ ]:
%%capture
!pip install dotwiz gdown

La librería Hugging Face genera unas advertencias que no aplican en nuestros desarrollos, por lo que se silencian.

In [ ]:
from transformers import logging

logging.set_verbosity_error()

## Inicialización de constantes

Definiremos al objeto `CONSTANTS` que contendrá todas las constantes a utilizar organizadas de forma jerárquica.

In [ ]:
from dotwiz import DotWiz
from re import compile
from math import log, floor
from logging import info

CONSTANTS = DotWiz({
    "DATASETS": {
        "PAN": {
            "PATHS": {
                "TRAINING": {
                    "CONVERSATIONS":
                    "inputs/pan-2012-training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml",
                    "SEXUAL_PREDATORS":
                    "inputs/pan-2012-training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt"
                },
                "TESTING": {
                    "CONVERSATIONS":
                    "inputs/pan-2012-testing/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml",
                    "SEXUAL_PREDATORS":
                    "inputs/pan-2012-testing/pan12-sexual-predator-identification-groundtruth-problem1.txt",
                    "SUSPICIOUS_CONVERSATIONS":
                    "inputs/pan-2012-testing/pan12-sexual-predator-identification-groundtruth-problem2.txt"
                }
            }
        }
    },
    "MODELS": {
        "BERT": {
            "NAME": "bert-base-uncased",
            "TOKENIZER": {
                "MAX_LENGTH": 128,
                "PADDING": "max_length",
                "TRUNCATION": True
            },
        },
        "32BS_24MBS_2e-05LR_3E": "outputs/phase_1/models/32BS_24MBS_2e-05LR_3E",
        "32BS_24MBS_5e-05LR_4E": "outputs/phase_1/models/32BS_24MBS_5e-05LR_4E"
    },
    "PREPROCESSING": {
        "RANDOMNESS": {
            "SEED": 400
        },
        "MESSAGE_BATCHING": {
            "SIZE": 24
        },
        "REGEXES": {
            "XML_QUOTE_ESCAPES": compile(r"&(apos|quot);")
        }
    },
    "INPUTS": {
        "FOCUS_MODEL": "32BS_24MBS_5e-05LR_4E"
    }
})

Para poder recrear (hasta cierto punto) cada uno de los resultados propuestos, preestableceremos una semilla. Es posible que existan diferencias entre una ejecución y otra ya que como trabajaremos a nivel de GPU, muchas de las operaciones en Tensorflow son procesadas de manera asíncrona, y muchos de los valores que tratamos acá requieren sumar flotantes que sí se ven afectados cuando cambian sus órdenes. Si quisiéramos habilitar un determinismo completo, usaríamos la instrucción `tensorflow.config.experimental.enable_op_determinism()`, pero veríamos una degradación en el desempeño de las instrucciones en varios órdenes de magnitud. Para mayores detalles, revisar la [documentación oficial](https://www.tensorflow.org/versions/r2.8/api_docs/python/tf/config/experimental/enable_op_determinism) de Tensorflow.

In [ ]:
from tensorflow.keras.utils import set_random_seed

set_random_seed(CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)

## Generación del conjunto de entrenamiento y validación

El conjunto de datos de **entrenamiento** PAN2012 tiene dos archivos de utilidad:

1. `pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt`: Enlista todos los identificadores de los autores que (se sabe) son depredadores sexuales separados por saltos de línea.
1. `pan12-sexual-predator-identification-training-corpus-2012-05-01.xml`: Enlista tanto conversaciones normales como pervertidas en un formato de etiquetas. Cada conversación se encierra con `<conversation>` y cada mensaje por `<message>`.

La cantidad de conversaciones normales versus pervertidas está altamente desequilibrada, por lo que se hará un tratamiento básico.

In [ ]:
from xml.etree import cElementTree as ET
from xml.sax.saxutils import unescape
from re import sub
from pandas import Series
from datasets import Dataset, concatenate_datasets
from transformers import DataCollatorWithPadding
from numpy.random import default_rng


def parse_xml_to_hf_dataset(sexual_predators_path, conversations_path):
    dataset = {
        "conversation_id": [],
        "conversation_label": [],
        "message": [],
    }

    with open(sexual_predators_path, "r") as file:
        sexual_predators = []

        for sexual_predator in file.readlines():
            sexual_predator = sexual_predator.strip()
            sexual_predators.append(sexual_predator)

    for event, element in ET.iterparse(conversations_path,
                                       events=("start", "end")):
        if event != "end" or element.tag != "conversation":
            continue

        conversation_id = element.get("id").strip()
        messages = element.findall("message")
        unescaped_messages = []

        conversation_includes_sexual_predator = False

        for index, message in enumerate(messages):
            author = message.find("author").text.strip()
            message = message.find("text")

            if message is None:
                continue

            if message.text is None:
                continue

            if author in sexual_predators:
                conversation_includes_sexual_predator = True

            unescaped_message = sub(
                CONSTANTS.PREPROCESSING.REGEXES.XML_QUOTE_ESCAPES, "'",
                unescape(message.text.strip()))
            unescaped_messages.append(unescaped_message)

        if not unescaped_messages:
            continue

        grouped_messages = []

        for index in range(0, len(unescaped_messages),
                           CONSTANTS.PREPROCESSING.MESSAGE_BATCHING.SIZE):
            grouped_messages.append("\n".join(
                unescaped_messages[index:index + CONSTANTS.PREPROCESSING.
                                   MESSAGE_BATCHING.SIZE]))

        dataset["conversation_id"].append(conversation_id)
        dataset["conversation_label"].append(
            conversation_includes_sexual_predator)
        dataset["message"].append(grouped_messages)

    hf_dataset = Dataset.from_dict(dataset)

    return hf_dataset

## Recuperación del modelo

Se recuperará el modelo `CONSTANTS.INPUTS.FOCUS_MODEL` entrenado durante la fase 1 y su respectivo *tokenizer*.

In [ ]:
from transformers import AutoTokenizer
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained(
    CONSTANTS.MODELS[CONSTANTS.INPUTS.FOCUS_MODEL])
tokenizer = AutoTokenizer.from_pretrained(CONSTANTS.MODELS.BERT.NAME)

## Generación de los conjuntos de datos

Obtendremos los *embeddings* de cada mensaje.

In [ ]:
from tensorflow import constant


def pooler_outputs(example, model):
    indexes = []
    flatten_input_ids = []

    for input_ids in example["input_ids"]:
        if len(indexes) == 0:
            indexes.append((0, len(input_ids)))
        else:
            indexes.append((indexes[-1][1], indexes[-1][1] + len(input_ids)))

        flatten_input_ids += input_ids

    tf_example = constant(flatten_input_ids)
    pooler_output = model.bert(tf_example).pooler_output
    reshaped_pooler_output = []

    for index in indexes:
        reshaped_pooler_output.append(pooler_output[index[0]:index[1]])

    return {"pooler_output": reshaped_pooler_output}


hf_training_dataset = parse_xml_to_hf_dataset(
    CONSTANTS.DATASETS.PAN.PATHS.TRAINING.SEXUAL_PREDATORS,
    CONSTANTS.DATASETS.PAN.PATHS.TRAINING.CONVERSATIONS)
hf_training_dataset = hf_training_dataset.map(lambda example: tokenizer(
    example["message"],
    max_length=CONSTANTS.MODELS.BERT.TOKENIZER.MAX_LENGTH,
    padding=CONSTANTS.MODELS.BERT.TOKENIZER.PADDING,
    truncation=CONSTANTS.MODELS.BERT.TOKENIZER.TRUNCATION))
hf_training_dataset = hf_training_dataset.map(
    lambda example: pooler_outputs(example, model),
    batched=True,
    batch_size=64)

hf_testing_dataset = parse_xml_to_hf_dataset(
    CONSTANTS.DATASETS.PAN.PATHS.TESTING.SEXUAL_PREDATORS,
    CONSTANTS.DATASETS.PAN.PATHS.TESTING.CONVERSATIONS)
hf_testing_dataset = hf_testing_dataset.map(lambda example: tokenizer(
    example["message"],
    max_length=CONSTANTS.MODELS.BERT.TOKENIZER.MAX_LENGTH,
    padding=CONSTANTS.MODELS.BERT.TOKENIZER.PADDING,
    truncation=CONSTANTS.MODELS.BERT.TOKENIZER.TRUNCATION))
hf_testing_dataset = hf_testing_dataset.map(
    lambda example: pooler_outputs(example, model),
    batched=True,
    batch_size=64)

Guardaremos el conjunto anterior.

In [ ]:
hf_training_dataset.save_to_disk(
    f"outputs/phase_1/datasets/with_attentions/training/huggingface/{CONSTANTS.INPUTS.FOCUS_MODEL}"
)
hf_testing_dataset.save_to_disk(
    f"outputs/phase_1/datasets/with_attentions/testing/huggingface/{CONSTANTS.INPUTS.FOCUS_MODEL}"
)